
# DuckDB + MovieLens — Starter Notebook (VS Code)

Este notebook foi preparado para:
- ligar a um ficheiro `.duckdb` local
- correr queries úteis em SQL
- (opcional) importar CSVs para dentro da base
- exportar resultados

> **Dica:** no VS Code, seleciona o kernel do teu `.venv` onde tens `duckdb` e `pandas` instalados.


In [1]:

#Se precisares de instalar dependências no teu venv, descomenta a linha abaixo.
%pip install duckdb pandas


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 7.4 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 9.3 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 9.2 MB/s  0:00:026m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [pandas]2m4/5 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb, pandas as pd
from pathlib import Path

DATA_DIR = Path("..") / "data" / "100k"
#movies_path

links_path = DATA_DIR / "links.csv"
movies_path = DATA_DIR / "movies.csv"
ratings_path = DATA_DIR / "ratings.csv"
tags_path = DATA_DIR / "tags.csv"

In [3]:
# cria ou liga se já existir:
con = duckdb.connect("movielens100K.duckdb")

# ver tabelas existentes
existing = set(con.sql("SHOW TABLES").df()["name"].str.lower())

# criar tabelas só se ainda não existirem
# movies (importação directa sem alterações — é texto e ok)
if "movies" not in existing:
    con.sql(f"CREATE TABLE movies AS SELECT * FROM read_csv_auto('{movies_path}')")

# ratings (na importação é importante garantir timestamp correcto)
con.sql(f"""
CREATE OR REPLACE TABLE ratings AS
SELECT
    userId::INT     As userId,
    movieId::INT    As movieId,
    rating::DOUBLE  As rating,
    to_timestamp(CAST(timestamp AS BIGINT)) AS timestamp
FROM read_csv_auto('{ratings_path}')
""")

# tags (também garantir timestamp)
con.sql(f"""
CREATE OR REPLACE TABLE tags AS
SELECT
    userId::INT     As userId,
    movieId::INT    As movieId,
    tag,
    to_timestamp(CAST(timestamp AS BIGINT)) AS timestamp
FROM read_csv_auto('{tags_path}')
""")

# links (tudo texto está ok)
if "links" not in existing:
    con.sql(f"CREATE TABLE links AS SELECT * FROM read_csv_auto('{links_path}')")

# mostrar estado final
con.sql("SHOW TABLES").df()

#Isto é melhor porquê?
#userId e movieId ficam logo como inteiros (não como texto)
#rating fica como número decimal real
#o timestamp passa a tipo TIMESTAMP (ts)


,name
0,links
1,movies
2,ratings
3,tags


## Fechar a ligação (quando terminares)

In [4]:
con.close()
print("Ligação fechada.")


Ligação fechada.
